<a href="https://colab.research.google.com/github/Ashishagrawa0503/Abalone-Age-prediction/blob/main/notebook256f767315.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os
import random
from PIL import Image, ImageEnhance
#pillow pli ek library hai jo python ka jo image preprocessig m help krta hai


#pillow pli ek library hai jo python ka jo image preprocessig m help krta hai

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Dense,Flatten,Dropout
from tensorflow.keras.preprocessing.image import load_img
#images ko load krne ke liye load images ka upyog liya gya hai
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
from sklearn.utils import shuffle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle dataset download -d masoudnickparvar/brain-tumor-mri-dataset
import zipfile
zip_ref = zipfile.ZipFile("/content/brain-tumor-mri-dataset.zip", 'r')
zip_ref.extractall("/content")
zip_ref.close()
train_dir="/content/Training"
test_dir="/content/Testing"
train_paths=[]
train_labels=[]
for label in os.listdir(train_dir):
  for image in os.listdir(os.path.join(train_dir,label)):#os.listdir(path) is a function from the os
  # module that returns a list containing the names of the entries in the directory given by path
    train_paths.append(os.path.join(train_dir,label,image))
    train_labels.append(label)

train_paths,train_labels=shuffle(train_paths,train_labels)
#image preprocessing
#-------------------

#image augmantation
def augemnt_image(image):
  image = Image.fromarray(np.uint8(image))#image ko pahle unsigned integer mai convert kiyafir Image.fromarry pillow fuchion
  #hai jiske help se image ko pillow formate main convert kiya kyoke hamne image enhancer pillow ka liya hua hai or wo pilow fatmate m image leta hai
  image=ImageEnhance.Brightness(image).enhance(random.uniform(0.8,1.2))
  #ImageEnhancer ke handar Brightness  call kiya or image diya fir is image ko enhance kiya par enhance karne k liye koi
  #ratio baatana padega is liye random uniform ka upyog kiya matlb image ka enhancement 0.80se 1.2ke beach mai rakhna hai
  #yadi image jo diya hai uske value 1.0 se aadhik hai to uske brightness kam krdo or yado 1.0 se less hai to image ke brightness badha do
  image=ImageEnhance.Contrast(image).enhance(random.uniform(0.8,1.2))
  image=ImageEnhance.Sharpness(image).enhance(random.uniform(0.8,1.2))
  image= np.array(image)/255.0
  #image ko pahle normalize kiya 0se 1 ke beach mai fr usko pillow formate se numpy formate mai convert kiya
  return image



#load image and apply augmentation
def open_images(paths):
  images = []
  for path in (paths):
    img= load_img(path,target_size(IMAGE_SIZE,IMAGE_SIZE))#SKlearn ka load image wala function jisko upper import kia hai
    img = augemnt_image(img)
    images.append(img)
  return np.array(images)# is images ke andar har pixel ka value hoge




#convert label name into integers
def encoder_label(labels):
  labels = os.listdir(train_dir)# jo 4 class the trasin dir ke andar usko 1,2,3,4 se encode kiya hai
  encoder = [labels.index(label) for label in labels]
  return encoder


#data generater for batching
def datagen(paths,labels,batch_size=12,epochs=1):
  for _ in range(epochs):
    for i in range(0,len(paths),batch_size):
      batch_paths = paths[i:i+batch_size]
      batch_labels = labels[i:i+batch_size]
      batch_images = open_images(batch_paths)
      batch_labels = encoder_label(batch_labels)
      yield batch_images,batch_labels




In [3]:
!kaggle datasets download -d masoudnickparvar/brain-tumor-mri-dataset

Dataset URL: https://www.kaggle.com/datasets/masoudnickparvar/brain-tumor-mri-dataset
License(s): CC0-1.0
 75% 112M/149M [00:00<00:00, 1.16GB/s]
100% 149M/149M [00:00<00:00, 982MB/s] 


4

In [8]:
IMAGE_SIZE=128
base_model=VGG16(weights="imagenet",include_top=False,input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))#(128*128 and RGB color)
#imagenet data ke liye jo weights the wo hamre model m be use ho


#freeze all layersof VGG16 base model(is model ke jitne be cnn rnn subko false kiya )
for layer in base_model.layers:
  layer.trainable=False

#set only last few layers(unfreeze some layers )

base_model.layers[-2].trainable=True
base_model.layers[-3].trainable=True
base_model.layers[-4].trainable=True


#bildmodel
model=Sequential()
model.add(Input(shape=(IMAGE_SIZE,IMAGE_SIZE,3)))#input layer and add VGG16
model.add(base_model)#image ka output multi dimension hoga is liye usko flatten kiya
model.add(Flatten)
model.(Dropout(0.3))
model.add(Dense(128,activation="relu"))#fully connection layer
model.add(Dropout(0.2))
model.add(Dense(len(os.listdir(train_dir)),activation="softmax"))

# compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),loss="categorical_crossentropy",metrics=["accuracy"])

#parameters
batch_size=20
steps=int(len(train_paths)/batch_size)
epochs=3

#train model
history=model.fit(
    datagen(train_paths,train_labels,batch_size=batch_size,epochs=epochs),
    steps_per_epoch=steps,
    epochs=epochs
)




SyntaxError: invalid syntax (<ipython-input-8-f831e7395077>, line 22)